# data

In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pathlib import Path
import calendar

# =========================
# Settings (2021-01-01 to 2022-06-30)
# =========================
START = "2021-01-01"
END   = "2023-06-30"

# Month starts: 2021-01-01 ... 2022-06-01
date_range = pd.date_range(start=START, end=END, freq="MS")

# Store GPT views
gpt_means, gpt_vars = [], []
dates = []            # month label (YYYY-MM-01)
sp500_tickers = None  # ticker universe (fixed from first month)

def _month_end(dt: pd.Timestamp) -> str:
    last_day = calendar.monthrange(dt.year, dt.month)[1]
    return dt.replace(day=last_day).strftime("%Y-%m-%d")

for current_date in date_range:
    start_date = current_date.strftime("%Y-%m-%d")
    end_date = _month_end(current_date)

    fp = Path(f"responses/gpt_{start_date}_{end_date}.json")
    if not fp.exists():
        raise FileNotFoundError(f"Missing GPT responses file: {fp}")

    gpt_dict = json.load(open(fp, "r"))

    # Fix ticker universe on first month (stable ordering)
    if sp500_tickers is None:
        sp500_tickers = list(gpt_dict.keys())

    means_row, vars_row = [], []
    for t in sp500_tickers:
        vals = (gpt_dict.get(t, {}) or {}).get("expected_return", None)
        if isinstance(vals, list) and len(vals) > 0:
            means_row.append(float(np.mean(vals)))
            vars_row.append(float(np.var(vals)))
        else:
            means_row.append(np.nan)
            vars_row.append(np.nan)

    gpt_means.append(means_row)
    gpt_vars.append(vars_row)
    dates.append(start_date)


In [2]:
# Views (means/vars) as DataFrames: rows=tickers, columns=month-start dates
gpt_means_df = pd.DataFrame(np.array(gpt_means), index=dates, columns=sp500_tickers).T
gpt_vars_df  = pd.DataFrame(np.array(gpt_vars),  index=dates, columns=sp500_tickers).T

In [3]:
import os
import glob
import pandas as pd

PORT_DIR = "responses_portfolios"

def _find_date_col(df: pd.DataFrame) -> str:
    for c in ["date_key", "Date", "date", "ym"]:
        if c in df.columns:
            return c
    # fallback: first unnamed index column
    if len(df.columns) > 0 and str(df.columns[0]).lower().startswith("unnamed"):
        df = df.rename(columns={df.columns[0]: "date_key"})
        return "date_key"
    raise ValueError(f"Aucune colonne de date trouvée. Colonnes: {list(df.columns)}")

def _read_and_standardize(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    date_col = _find_date_col(df)

    # Standardize to 'date_key' as datetime
    if date_col == "ym":
        # 'YYYY-MM' -> month start
        df["date_key"] = pd.to_datetime(df["ym"].astype(str) + "-01", errors="coerce")
    else:
        df["date_key"] = pd.to_datetime(df[date_col], errors="coerce")

    # drop original date cols (except date_key)
    drop_cols = [c for c in ["Date", "date", "ym"] if c in df.columns and c != "date_key"]
    df = df.drop(columns=drop_cols, errors="ignore")

    # keep only rows with date
    df = df.dropna(subset=["date_key"]).copy()
    return df

def combine_monthly_portfolios(
    pattern: str,
    out_path: str,
) -> pd.DataFrame:
    files = sorted(glob.glob(os.path.join(PORT_DIR, pattern)))
    if not files:
        raise FileNotFoundError(f"Aucun fichier trouvé avec le pattern: {PORT_DIR}/{pattern}")

    dfs = []
    for f in files:
        df = _read_and_standardize(f)
        df["source_file"] = os.path.basename(f)  # pratique pour debug
        dfs.append(df)

    out = pd.concat(dfs, ignore_index=True)

    # sort & drop duplicates by date (keep last)
    out = out.sort_values("date_key")
    out = out.drop_duplicates(subset=["date_key"], keep="last").reset_index(drop=True)

    # save combined
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    out.to_csv(out_path, index=False)
    return out

# ---- Build combined baseline files ----
eq_combined = combine_monthly_portfolios(
    pattern="equal_weighted_portfolio*.csv",
    out_path=os.path.join(PORT_DIR, "equal_weighted_portfolio.csv"),
)

opt_combined = combine_monthly_portfolios(
    pattern="optimized_portfolio*.csv",
    out_path=os.path.join(PORT_DIR, "optimized_portfolio.csv"),
)

print("Baselines combinés :")
print(" -", os.path.join(PORT_DIR, "equal_weighted_portfolio.csv"), "(", len(eq_combined), "rows )")
print(" -", os.path.join(PORT_DIR, "optimized_portfolio.csv"), "(", len(opt_combined), "rows )")
display(eq_combined.head())
display(opt_combined.head())


Baselines combinés :
 - responses_portfolios\equal_weighted_portfolio.csv ( 30 rows )
 - responses_portfolios\optimized_portfolio.csv ( 30 rows )


,Portfolio_Return,date_key,source_file
0,0.061814,2021-02-01,equal_weighted_portfolio_2021-01-01_2021-01-31...
1,0.058395,2021-03-01,equal_weighted_portfolio_2021-02-01_2021-02-28...
2,0.046023,2021-04-01,equal_weighted_portfolio_2021-03-01_2021-03-31...
3,0.018338,2021-05-01,equal_weighted_portfolio_2021-04-01_2021-04-30...
4,0.000543,2021-06-01,equal_weighted_portfolio_2021-05-01_2021-05-31...


,Portfolio_Return,date_key,source_file
0,0.061814,2021-02-01,optimized_portfolio_2021-01-01_2021-01-31.csv
1,0.058395,2021-03-01,optimized_portfolio_2021-02-01_2021-02-28.csv
2,0.046023,2021-04-01,optimized_portfolio_2021-03-01_2021-03-31.csv
3,0.018338,2021-05-01,optimized_portfolio_2021-04-01_2021-04-30.csv
4,0.000543,2021-06-01,optimized_portfolio_2021-05-01_2021-05-31.csv


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

"""
Baselines portfolio returns (generated by baselines.py)
Expected files:
  - responses_portfolios/equal_weighted_portfolio.csv
  - responses_portfolios/optimized_portfolio.csv
"""
equal_weighted_portfolio_returns = pd.read_csv('responses_portfolios/equal_weighted_portfolio.csv')
optimized_portfolio_returns      = pd.read_csv('responses_portfolios/optimized_portfolio.csv')

equal_weighted_portfolio_returns.head()


,Portfolio_Return,date_key,source_file
0,0.061814,2021-02-01,equal_weighted_portfolio_2021-01-01_2021-01-31...
1,0.058395,2021-03-01,equal_weighted_portfolio_2021-02-01_2021-02-28...
2,0.046023,2021-04-01,equal_weighted_portfolio_2021-03-01_2021-03-31...
3,0.018338,2021-05-01,equal_weighted_portfolio_2021-04-01_2021-04-30...
4,0.000543,2021-06-01,equal_weighted_portfolio_2021-05-01_2021-05-31...


In [5]:
"""
GPT monthly portfolio weights (Black-Litterman output from evaluate_multiple_updated.py)
"""
import pandas as pd
import numpy as np

tau = 0.025

weights_path = f"results/gpt_black_litterman_weights_tau_{tau}.csv"
gpt_weights_df = pd.read_csv(weights_path)

# detect date column
if "Date" in gpt_weights_df.columns:
    date_col = "Date"
elif "date_key" in gpt_weights_df.columns:
    date_col = "date_key"
else:
    date_col = gpt_weights_df.columns[0]

gpt_weights_df[date_col] = pd.to_datetime(gpt_weights_df[date_col], errors="coerce")
gpt_weights_df = gpt_weights_df.dropna(subset=[date_col]).set_index(date_col).sort_index()

# transpose for plotting: rows=tickers, columns=dates
gpt_results = gpt_weights_df.T


FileNotFoundError: [Errno 2] No such file or directory: 'results/gpt_black_litterman_weights_tau_0.025.csv'

In [ ]:
"""
Gemma3 1B monthly portfolio weights (Black-Litterman output from evaluate_multiple.py)
"""
import pandas as pd
import numpy as np

tau = 0.025

weights_path = f"results/gemma3_1b_black_litterman_weights_tau_{tau}.csv"
gemma_weights_df = pd.read_csv(weights_path)

# detect date column
if "Date" in gemma_weights_df.columns:
    date_col = "Date"
elif "date_key" in gemma_weights_df.columns:
    date_col = "date_key"
else:
    date_col = gemma_weights_df.columns[0]

gemma_weights_df[date_col] = pd.to_datetime(gemma_weights_df[date_col], errors="coerce")
gemma_weights_df = gemma_weights_df.dropna(subset=[date_col]).set_index(date_col).sort_index()

# transpose for plotting: rows=tickers, columns=dates
gemma_results = gemma_weights_df.T


In [ ]:
"""
Qwen2.5 1.5B monthly portfolio weights (Black-Litterman output)
"""
import glob
import pandas as pd
import numpy as np

tau = 0.025

# Try a couple of common naming conventions
patterns = [
    f"results/qwen_black_litterman_weights_tau_{tau}.csv",
    f"results/qwen_black_litterman_weights_tau_{tau}_*.csv",
]

candidates = []
for pat in patterns:
    candidates += sorted(glob.glob(pat))

# If both exact + wildcard matched, keep unique
candidates = sorted(set(candidates))
if not candidates:
    raise FileNotFoundError(
        "Could not find Qwen weights CSV in results/. Tried patterns:"
        + "".join(f"- {p}" for p in patterns)
        + "Generate it first (e.g., evaluate_multiple / calculate_llm_returns)."
    )

qwen_weights_path = candidates[-1]
print("Using:", qwen_weights_path)
qwen_weights_df = pd.read_csv(qwen_weights_path)

# detect date column
if "Date" in qwen_weights_df.columns:
    date_col = "Date"
elif "date_key" in qwen_weights_df.columns:
    date_col = "date_key"
else:
    date_col = qwen_weights_df.columns[0]

qwen_weights_df[date_col] = pd.to_datetime(qwen_weights_df[date_col], errors="coerce")
qwen_weights_df = qwen_weights_df.dropna(subset=[date_col]).set_index(date_col).sort_index()

# transpose for plotting: rows=tickers, columns=dates
qwen_results = qwen_weights_df.T
qwen_results.head()


In [ ]:
"""
GPT portfolio returns (generated by calculate_llm_returns.py)
"""
import glob
import pandas as pd

tau = 0.025

# Match: results/gpt_black_litterman_returns_tau_0.025_2021-01-01_2022-06-30.csv
pattern1 = f"results/gpt_black_litterman_returns_tau_{tau}_*.csv"
pattern2 = f"results/gpt_black_litterman_returns_tau_{tau}.csv"

candidates = sorted(glob.glob(pattern1))
if not candidates:
    candidates = sorted(glob.glob(pattern2))

if not candidates:
    raise FileNotFoundError(
        f"Could not find GPT returns CSV in results/. Tried patterns:\n- {pattern1}\n- {pattern2}\n"
        "Run calculate_llm_returns.py first."
    )

gpt_returns_path = candidates[-1]
print("Using:", gpt_returns_path)
gpt_returns = pd.read_csv(gpt_returns_path)
gpt_returns.head()


In [ ]:
"""
Gemma3 1B portfolio returns (generated by calculate_llm_returns.py)
"""
import glob
import pandas as pd

tau = 0.025

candidates = sorted(glob.glob(f"results/gemma3_1b_black_litterman_returns_tau_{tau}_*.csv"))
if not candidates:
    candidates = sorted(glob.glob(f"results/gemma3_1b_black_litterman_returns_tau_{tau}.csv"))

if not candidates:
    raise FileNotFoundError("Could not find Gemma3_1b returns CSV in results/. Run calculate_llm_returns.py first.")

gemma_returns_path = candidates[-1]
print("Using:", gemma_returns_path)
gemma_returns = pd.read_csv(gemma_returns_path)
gemma_returns.head()


In [ ]:
"""
Qwen2.5 1.5B portfolio returns (generated by calculate_llm_returns.py)
"""
import glob
import pandas as pd

tau = 0.025

pattern1 = f"results/qwen_black_litterman_returns_tau_{tau}_*.csv"
pattern2 = f"results/qwen_black_litterman_returns_tau_{tau}.csv"

candidates = sorted(glob.glob(pattern1))
if not candidates:
    candidates = sorted(glob.glob(pattern2))

if not candidates:
    raise FileNotFoundError(
        f"Could not find Qwen returns CSV in results/. Tried patterns:- {pattern1}- {pattern2}"
        "Run calculate_llm_returns.py first."
    )

qwen_returns_path = candidates[-1]
print("Using:", qwen_returns_path)
qwen_returns = pd.read_csv(qwen_returns_path)
qwen_returns.head()


In [ ]:
"""
S&P 500 proxy returns (monthly) from yfinance ETF (e.g., SPY):
- Download daily adjusted prices
- Convert to monthly returns (month-end to month-end)
- Keep Jan..Jun 2023 inclusive
"""
import pandas as pd

try:
    import yfinance as yf
except ImportError:
    raise ImportError("yfinance n'est pas installé. Fais: pip install yfinance")

ETF = "SPY"  # alternatives: "IVV", "VOO"

px = yf.download(ETF, start=START, end=END, progress=False)
if px.empty:
    raise ValueError(f"No data downloaded for {ETF}. Check internet access / ticker.")

# Prefer Adj Close if available, else Close
if "Adj Close" in px.columns:
    close = px["Adj Close"]
else:
    close = px["Close"]

# If DataFrame (edge case), take first column
if isinstance(close, pd.DataFrame):
    close = close.iloc[:, 0]

close = close.dropna()

# Month-end close and monthly returns
m_close = close.resample("ME").last()
m_ret = m_close.pct_change().dropna()

sp500_proxy = pd.DataFrame({
    "date_key": m_ret.index,
    "Portfolio_Return": m_ret.values
})

# Keep months end Jan..Jun 2023
sp500_proxy = sp500_proxy[(sp500_proxy["date_key"] >= START) & (sp500_proxy["date_key"] <= END)].copy()
sp500_proxy


# evaluate

In [ ]:
import numpy as np
import pandas as pd

def _standardize_returns(df: pd.DataFrame, name: str) -> pd.Series:
    """
    Accepts df with 'Portfolio_Return' and one date column among: date_key / Date / ym.
    Returns a Series indexed by datetime at MONTH START (YYYY-MM-01).
    """
    df = df.copy()

    # Detect date column
    if "date_key" in df.columns:
        dt = pd.to_datetime(df["date_key"], errors="coerce")
    elif "Date" in df.columns:
        dt = pd.to_datetime(df["Date"], errors="coerce")
    elif "ym" in df.columns:
        dt = pd.to_datetime(df["ym"].astype(str) + "-01", errors="coerce")
    else:
        raise ValueError(f"{name}: could not find a date column in {list(df.columns)}")

    if "Portfolio_Return" not in df.columns:
        raise ValueError(f"{name}: missing 'Portfolio_Return' column")

    s = pd.Series(pd.to_numeric(df["Portfolio_Return"], errors="coerce").values, index=dt)
    s = s.dropna()

    # Normalize to month start
    s.index = s.index.to_period("M").to_timestamp()

    # Deduplicate and sort
    s = s[~s.index.duplicated(keep="last")].sort_index()
    return s

def _infer_periods_per_year(index: pd.DatetimeIndex) -> int:
    # For monthly series
    return 12

# Build return series
r_sp500 = _standardize_returns(sp500_proxy, "SP500 proxy")
r_ew    = _standardize_returns(equal_weighted_portfolio_returns, "EW")
r_mvo   = _standardize_returns(optimized_portfolio_returns, "MVO")
r_gpt   = _standardize_returns(gpt_returns, "BLM-GPT")
r_gemma = _standardize_returns(gemma_returns, "BLM-Gemma3_1b")
r_qwen  = _standardize_returns(qwen_returns, "BLM-Qwen2.5_1.5b")

# Align by common dates
common_idx = (r_sp500.index
    .intersection(r_ew.index)
    .intersection(r_mvo.index)
    .intersection(r_gpt.index)
    .intersection(r_gemma.index)
    .intersection(r_qwen.index))

r_sp500 = r_sp500.loc[common_idx]
r_ew    = r_ew.loc[common_idx]
r_mvo   = r_mvo.loc[common_idx]
r_gpt   = r_gpt.loc[common_idx]
r_gemma = r_gemma.loc[common_idx]
r_qwen  = r_qwen.loc[common_idx]

portfolios = {
    "SP500": r_sp500,
    "EW": r_ew,
    "MVO": r_mvo,
    "BLM-GPT": r_gpt,
    "BLM-Gemma3_1b": r_gemma,
    "BLM-Qwen2.5_1.5b": r_qwen,
}

ppy = _infer_periods_per_year(common_idx)
rf = 0.02 / ppy  # per-period risk-free rate (2% annual)

stats = {}
for name, returns in portfolios.items():
    returns = returns.astype(float)
    cum = (1 + returns).prod() - 1
    ann_ret = (1 + cum) ** (ppy / max(1, len(returns))) - 1
    ann_vol = returns.std(ddof=1) * np.sqrt(ppy)
    sharpe = ((returns.mean() - rf) / returns.std(ddof=1)) * np.sqrt(ppy) if returns.std(ddof=1) > 0 else np.nan

    wealth = (1 + returns).cumprod()
    peak = wealth.cummax()
    dd = wealth / peak - 1
    mdd = dd.min()

    stats[name] = {
        "Cum. Return": round(float(cum), 4),
        "Ann. Return": round(float(ann_ret), 4),
        "Ann. Vol": round(float(ann_vol), 4),
        "Sharpe": round(float(sharpe), 4),
        "Max Drawdown": round(float(mdd), 4),
    }

stats_df = pd.DataFrame(stats).T
print(f"Periods/year inferred: {ppy}")
stats_df


In [ ]:
import matplotlib.pyplot as plt
import matplotlib

plt.figure(figsize=(12, 6))

for name, returns in portfolios.items():
    cum = (1 + returns).cumprod()
    plt.plot(cum.index, cum.values, label=name)

plt.grid(True, linestyle="--", alpha=0.7)
plt.xlabel("Date", fontsize=13)
plt.ylabel("Cumulative Return", fontsize=13)
plt.legend(loc="upper left", fontsize=11)

plt.gca().xaxis.set_major_locator(matplotlib.dates.MonthLocator())
plt.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y-%m"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()
